In [11]:
# with open("test_files.txt", 'r') as file:
#     lines = file.readlines()

# parsed_data = [(int(line.split()[0]), line.split()[1]) for line in lines]

# TODO: Add save pc_idx, screen_coords 
# io = IO()
# for file, category in parsed_data:   
#     xyz, rgb = normalize_pc(f"data/{category}/{file}/pc.ply", f'examples/zeroshot_{category}', io, device) #read Point cloud and rgb in the format n,3
#     img_dir, pc_idx, screen_coords = render_pc(xyz, rgb, f'examples/zeroshot_{category}', device, file)
#     
#     np.savez( f'examples/zeroshot_{category}/rendered_img_test/{file}/pcidx_screencoords.npz', 
#              pc_idx=pc_idx, 
#              screen_coords=screen_coords)

In [12]:
import sys
sys.path.insert(0,'../GroundingDINO')
sys.path.insert(0,'../SAM')
import os
import torch
import json
from pytorch3d.io import IO
import numpy as np
from src.utils import normalize_pc,save_colored_pc
from src.render_pc import render_pc
from src.gen_superpoint import gen_superpoint
from groundingdino.util.inference import load_model, load_image, predict, annotate
import cv2
from torchvision.ops import nms
import json
from segment_anything import sam_model_registry, SamPredictor
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors
import distinctipy

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [13]:
def yolobbox2bbox(yolobox):
    x = yolobox[:,0]
    y = yolobox[:,1]
    w = yolobox[:,2]
    h = yolobox[:,3]
    xyxy = np.zeros_like(yolobox)
    xyxy[:,0] = x-w/2
    xyxy[:,1] = y-h/2
    xyxy[:,2] = x+w/2
    xyxy[:,3] = y+h/2
    return xyxy

def toDinoPrompt(metaData,className):
    listOfParts = metaData[className]
    prompt = ""
    partList = {}
    for i,part in enumerate(listOfParts):
        prompt += f"{part} of {className}.".lower()
        partList[f"{part}".lower()] = i
    return prompt,partList

def check_pc_within_bbox(x1, y1, x2, y2, pc):  
    flag = np.logical_and(pc[:, 0] > x1, pc[:, 0] < x2)
    flag = np.logical_and(flag, pc[:, 1] > y1)
    flag = np.logical_and(flag, pc[:, 1] < y2)
    return flag

def InferDINO(xyz, rgb,pc_idx, screen_coords, file,category,metaData,modelDINO,predictorSAM,device,BOX_TRESHOLD = 0.2, TEXT_TRESHOLD = 0.3, SCORE_THRESHOLD=0.2,n_neighbors=21,n_pass=3, save_dir="tmp"):
    TEXT_PROMPT,partList = toDinoPrompt(metaData,category)
    
    code = str(file.split("/")[-1])
    
    preds = []
    for i in range(pc_idx.shape[0]):
        image_source, image = load_image(f"{save_dir}/rendered_img_test/{code}/{i}.png") #load rgb images
        predictorSAM.set_image(image_source)

        boxes, logits, phrases = predict(
                                        model=modelDINO,
                                        image=image,
                                        caption=TEXT_PROMPT,
                                        box_threshold=BOX_TRESHOLD,
                                        text_threshold=TEXT_TRESHOLD
                                    )
        phrases = np.array(phrases) #just to fix indexing

        xyxy = yolobbox2bbox(boxes)*image.shape[-1] #change bbox format to xyxy and scale with image size
        
        nms_mask = []
        for t,bbox in enumerate(xyxy): 
            if check_pc_within_bbox(bbox[0], bbox[1], bbox[2], bbox[3], screen_coords[i]).mean() < 0.95: 
                nms_mask.append(t)
                
        xyxy = xyxy[nms_mask]
        boxes = boxes[nms_mask]
        logits = logits[nms_mask]
        phrases = phrases[nms_mask]
        
        
        
        nms_indexes = nms(torch.tensor(xyxy) , logits, 0.5).numpy() #non maximum supression
        
        xyxy = xyxy[nms_indexes]
        boxes = boxes[nms_indexes]
        logits = logits[nms_indexes]
        phrases = phrases[nms_indexes]
        
        final_indexes = []
        new_phrases = []
        for iindex,part in enumerate(phrases):
            for metaPart in metaData[category]:
                if part.find(metaPart) != -1:
                    final_indexes.append(iindex)
                    new_phrases.append(metaPart)
                    
        
        xyxy = xyxy[final_indexes]
        boxes = boxes[final_indexes]
        logits = logits[final_indexes]
        phrases = np.array(new_phrases)
        #preds.append({"image_source":image_source,"xyxy":xyxy[final_indexes],"boxes":boxes[final_indexes],"logits":logits[final_indexes],"phrases":new_phrases})
        
        input_boxes = torch.tensor(xyxy, device=predictorSAM.device)    
        transformed_boxes = predictorSAM.transform.apply_boxes_torch(input_boxes, image_source.shape[:2])
        

        if transformed_boxes.shape[0] > 0:
            masks, _, _ = predictorSAM.predict_torch(
                point_coords=None,
                point_labels=None,
                boxes=transformed_boxes,
                multimask_output=False,
            )    #create segmentation masks with sam

            for index,mask in enumerate(masks):
#                 print(mask[1:].shape)
                preds.append({'image_id': i, 'category_id': phrases[index], 
                              'bbox': boxes[index]*image.shape[-1], 
                              'score': logits[index],
                              'mask':mask[0],
                              'image_source':image_source
                             }
                            )
    pc_aggMask = torch.zeros((xyz.shape[0],len(partList)+1)) #this is a segment agg mask we sum all the scores from our bboxes 
    #into their own respective channel, the last channel is for unsegmented parts
    pc_aggMask[:,-1] = SCORE_THRESHOLD #we can set a confidence threshold by setting the unsegmented score
    for prediction in preds:
        maskedPC_idx = pc_idx[prediction["image_id"],prediction["mask"].cpu().numpy()] #this gives you the pc idx of the points that are inside the mask
        index_pcMasked = np.unique(maskedPC_idx)[1:] # we only need the unique idx and the first id is always -1 meaning not found
        pc_aggMask[index_pcMasked,partList[prediction["category_id"]]] += prediction["score"] #add up all the scores for each part
    pc_seg_classes = torch.argmax(pc_aggMask,dim=-1) #select the highest score as our segmentation class
    #if non of the part scores are over the SCORE_THRESHOLD it will be left unsegmented
    partColors = np.array(distinctipy.get_colors(len(partList)))
    accumulator = np.zeros((xyz.shape[0], len(partList)+1))
    accumulator[:,-1] = 1
    # since projections are not exact meaning not every PC point is rendered into our image our backprojections are not dense
    # use KNN to smooth these backprojections 
    nn = NearestNeighbors(n_neighbors=n_neighbors, algorithm='kd_tree').fit(xyz) #create a knn
    
    results = {"partseg_rgbs":{}}
    
    os.makedirs(f"{save_dir}/semantic_segDino_KNN/{code}", exist_ok=True)
    for colorId,part in enumerate(partList):
        pc_part_idx = np.zeros((xyz.shape[0]),dtype=int)
        rgb_sem = np.zeros((xyz.shape[0],3))
        pc_part_idx[torch.where(pc_seg_classes==partList[part])] = 1
        
        for pass_ in range(n_pass):
            notColoredIndexes = torch.where(pc_seg_classes!=partList[part]) #find non segmented parts for smoothing

            n_indexes = nn.kneighbors(xyz[notColoredIndexes],n_neighbors+1,return_distance=False)
            n_indexes = n_indexes[:,1:] #get n_neighbors for the points, the first index is always the point itself so delete that
            #we have dense point clouds so distance based measures are not necessary and sometimes give worst results
            flag = pc_part_idx[n_indexes].mean(axis=1) 
            
            flag[np.where(flag>=0.4)] = 1 #and segmnent the points where the mean of neighbours are colored %40 or over
            flag[np.where(flag<0.4)] = 0
            pc_part_idx[notColoredIndexes] = flag
           
        rgb_sem[pc_part_idx.astype(bool)] = partColors[colorId]
        accumulator[pc_part_idx.astype(bool),colorId] += 1
        save_colored_pc(f"{save_dir}/semantic_segDino_KNN/{code}/{part}.ply", xyz, rgb_sem)
        results["partseg_rgbs"][part] = rgb_sem
        
    pc_partIDX = np.argmax(accumulator,axis=-1)
    partColors_extended = np.append(partColors,[[0,0,0]],axis=0)
    
    save_colored_pc(f"{save_dir}/semantic_segDino_KNN/{code}/{category}.ply", xyz, partColors_extended[pc_partIDX.astype(int)]) 
    
    results["partList"] = partList
    results["xyz"] = xyz
    return results

In [14]:
metaData = json.load(open("./PartNetE_meta.json")) 
device = "cpu"
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    torch.cuda.set_device(device)
modelDINO = load_model("../GroundingDINO/groundingdino/config/GroundingDINO_SwinB_cfg.py",
                    "../GroundingDINO/weights/groundingdino_swinb_cogcoor.pth",
                    device=device
                    )
sam_checkpoint = "../SAM/weights/sam_vit_h_4b8939.pth"
model_type = "vit_h"


sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device="cuda")

predictorSAM = SamPredictor(sam)

final text_encoder_type: bert-base-uncased


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [15]:
def get_files_from_txt(file_path, categories, num_files):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    
    parsed_data = [(int(line.split()[0]), line.split()[1]) for line in lines]
    
    results = {}
    for category in categories:
        filtered_data = [f"data/{category}/{item[0]}" for item in parsed_data if item[1] == category]
        results[category] = filtered_data[:num_files]
    
    return results

In [16]:
def mIOU_f(label, predictions, case='dino', verbose=True):
    """
    Calculate mean IoU for semantic segmentation predictions.

    Args:
        label: Ground truth label containing 'semantic_seg'.
        predictions: Predictions dictionary from the model.
        case: Specify 'dino' or 'glip' for the respective models.
        verbose: If True, prints detailed IoU calculation logs.

    Returns:
        Tuple of (list of IoUs, mean IoU).
    """
    def log(msg):
        if verbose:
            print(msg)

    try:
        # Validate ground truth (label)
        if 'semantic_seg' not in label.item():
            raise ValueError("The label does not contain 'semantic_seg'.")
        semantic_seg = np.array(label.item()['semantic_seg'])
        unique_parts = np.unique(semantic_seg)
        unique_parts = unique_parts[unique_parts != -1]  # Exclude background (-1)

        ious = []
        part_list_return = []
        part_names = None
        if case == 'dino':
            # Validate predictions for 'dino' case
            if 'partList' not in predictions or 'partseg_rgbs' not in predictions:
                raise ValueError(
                    "Invalid 'predictions' format for 'dino'. Expected keys: 'partList' and 'partseg_rgbs'."
                )
            part_list = predictions['partList']
            partseg_rgbs = predictions['partseg_rgbs']

            for part, part_id in part_list.items():
                if part_id not in unique_parts:
                    log(f"{part} not present in ground truth, skipping IoU calculation.")
                    continue
                
                part_list_return.append(part)
                gt_mask = semantic_seg == part_id
                pred_mask = np.any(partseg_rgbs[part] != [0., 0., 0.], axis=-1)

                intersection = np.logical_and(gt_mask, pred_mask).sum()
                union = np.logical_or(gt_mask, pred_mask).sum()

                iou = intersection / union if union > 0 else 0
                ious.append(iou)
                log(f"IoU for {part}: {iou:.4f}")

        elif case == 'glip':
            # Validate predictions for 'other' case
            if 'part_names_ordered' not in predictions or 'sem_seg' not in predictions:
                raise ValueError(
                    "Invalid 'predictions' format for 'glip'. Expected keys: 'part_names_ordered' and 'sem_seg'."
                )
            part_names = predictions['part_names_ordered']
            sem_seg_pred = np.array(predictions['sem_seg'])
            category = str(predictions['category'])
            
            for idx, part_name in enumerate(part_names):
                if idx not in unique_parts:
                    log(f"{category.lower()} {part_name} not present in ground truth, skipping IoU calculation.")
                    continue
                gt_mask = semantic_seg == idx
                pred_mask = sem_seg_pred == idx

                intersection = np.logical_and(gt_mask, pred_mask).sum()
                union = np.logical_or(gt_mask, pred_mask).sum()

                iou = intersection / union if union > 0 else 0
                ious.append(iou)
                log(f"IoU for {part_name}: {iou:.4f}")

        else:
            # Raise an error for unsupported cases
            raise ValueError(f"Unsupported case: {case}. Use 'dino' or 'glip'.")

        # Calculate mean IoU
        if ious:
            mean_iou = np.mean(ious)
            log(f"Mean IoU: {mean_iou:.4f}")
        else:
            log("No valid parts found in the ground truth. Mean IoU cannot be calculated.")
            mean_iou = None

        return ious, mean_iou, part_names

    except Exception as e:
        raise RuntimeError(f"Error calculating IoU: {e}") from e


In [17]:
from demo import InferGLIP

from src.glip_inference import load_model_glip

[nltk_data] Downloading package punkt to
[nltk_data]     /rhome/kamburoglu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /rhome/kamburoglu/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [ ]:
categories = ["Chair", "Suitcase", "Refrigerator", "Lamp", "Kettle"]

files = get_files_from_txt("test_files.txt", categories, 10)

config ="GLIP/configs/glip_Swin_L.yaml"
weight_path = "models/glip_large_model.pth"
#     print("[loading GLIP model...]")
glip_demo = load_model_glip(config, weight_path)


results = {}
# Comment out the two loop for the category and number assignment above
# for category, file_items in files.items():
with open("test_files.txt", 'r') as file:
    lines = file.readlines()

    parsed_data = [(int(line.split()[0]), line.split()[1]) for line in lines]
    
for file, category in parsed_data:
    file = f"data/{category}/{file}"
    print(category +" inference DINO, GLIP ==> SAM started" +":")
    #for file in file_items:
    TEXT_PROMPT,partList = toDinoPrompt(metaData, category)

    save_dir = f'examples/zeroshot_{category}'
    os.makedirs(save_dir, exist_ok=True)
    os.makedirs(f"{save_dir}/rendered_img", exist_ok=True) #create the necessary save directories
    os.makedirs(f"{save_dir}/dino_pred", exist_ok=True)
    os.makedirs(f"{save_dir}/semantic_segDino_KNN", exist_ok=True)

    io = IO()
    xyz, rgb = normalize_pc(file + "/pc.ply", save_dir, io, device) #read Point cloud and rgb in the format n,3

    code = str(file.split("/")[-1])
    pcidx_screencoords = np.load(f'examples/zeroshot_{category}/rendered_img_test/{code}/pcidx_screencoords.npz')

    pc_idx = pcidx_screencoords["pc_idx"]
    screen_coords = pcidx_screencoords["screen_coords"]

    print("----DINO Inference Starting----")
    preds = InferDINO(xyz, rgb, pc_idx, screen_coords, file, category, metaData,modelDINO,predictorSAM, device, BOX_TRESHOLD = 0.2,
                        TEXT_TRESHOLD = 0.3, save_dir=f'examples/zeroshot_{category}')

    print("----DINO Inference Completed---")
    print("mIOU CALCULATION DINO, " + category + ":")
    label = np.load(file + "/label.npy", allow_pickle=True)
    mIOU_list, mean_iou, _  = mIOU_f(label, preds)

    print("----GLIP Inference Starting----")
    preds_glip = InferGLIP(xyz, rgb, screen_coords, pc_idx, category, metaData[category], glip_demo, device, 
                           f"examples/zeroshot_{category}/rendered_img_test/{code}/")
    print("----GLIP Inference Completed---")

    print("mIOU CALCULATION GLIP, " + category + ":")
    mIOU_list_glip, mean_iou_glip, part_names = mIOU_f(label, preds_glip, "glip")
    results.setdefault(category, {}).setdefault(code, {
        'DINO': {
            'mIOU_list': mIOU_list,
            'mean_iou': mean_iou,
        },
        'GLIP': {
            'mIOU_list': mIOU_list_glip,
            'mean_iou': mean_iou_glip,
        }, 
        "parts": part_names

    })
    
    with open('results.json', 'w') as f:
        json.dump(results, f, indent=4)

    print("Results saved to results.json")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'bert.pooler.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
An exception occurred in telemetry logging.Disabling telemetry to prevent further exceptions.
Traceba

Bottle inference DINO, GLIP ==> SAM started:
----DINO Inference Starting----


----DINO Inference Completed---
mIOU CALCULATION DINO, Bottle:
IoU for lid: 0.8678
Mean IoU: 0.8678
----GLIP Inference Starting----


An exception occurred in telemetry logging.Disabling telemetry to prevent further exceptions.
Traceback (most recent call last):
  File "/rhome/kamburoglu/miniconda3/envs/partslip/lib/python3.9/site-packages/iopath/common/file_io.py", line 946, in __log_tmetry_keys
    handler.log_event()
  File "/rhome/kamburoglu/miniconda3/envs/partslip/lib/python3.9/site-packages/iopath/common/event_logger.py", line 97, in log_event
    del self._evt
AttributeError: _evt


----GLIP Inference Completed---
mIOU CALCULATION GLIP, Bottle:
IoU for lid: 0.8786
Mean IoU: 0.8786
Results saved to results.json
Bottle inference DINO, GLIP ==> SAM started:
----DINO Inference Starting----


...

----GLIP Inference Completed---
mIOU CALCULATION GLIP, Chair:
IoU for arm: 0.9830
IoU for back: 0.9668
IoU for leg: 0.6721
IoU for seat: 0.9810
IoU for wheel: 0.7984
Mean IoU: 0.8803
Results saved to results.json
Chair inference DINO, GLIP ==> SAM started:
----DINO Inference Starting----


----DINO Inference Completed---
mIOU CALCULATION DINO, Chair:
arm not present in ground truth, skipping IoU calculation.
IoU for back: 0.9970
IoU for leg: 0.6537
IoU for seat: 0.9556
IoU for wheel: 0.8676
Mean IoU: 0.8685
----GLIP Inference Starting----


An exception occurred in telemetry logging.Disabling telemetry to prevent further exceptions.
Traceback (most recent call last):
  File "/rhome/kamburoglu/miniconda3/envs/partslip/lib/python3.9/site-packages/iopath/common/file_io.py", line 946, in __log_tmetry_keys
    handler.log_event()
  File "/rhome/kamburoglu/miniconda3/envs/partslip/lib/python3.9/site-packages/iopath/common/event_logger.py", line 97, in log_event
    del self._evt
AttributeError: _evt


----GLIP Inference Completed---
mIOU CALCULATION GLIP, Chair:
chair arm not present in ground truth, skipping IoU calculation.
IoU for back: 0.9065
IoU for leg: 0.6722
IoU for seat: 0.8679
IoU for wheel: 0.9616
Mean IoU: 0.8521
Results saved to results.json
Chair inference DINO, GLIP ==> SAM started:
----DINO Inference Starting----


----DINO Inference Completed---
mIOU CALCULATION DINO, Chair:
IoU for arm: 0.3278
IoU for back: 0.5419
IoU for leg: 0.8864
IoU for seat: 0.7966
IoU for wheel: 0.9473
Mean IoU: 0.7000
----GLIP Inference Starting----
